<a href="https://colab.research.google.com/github/harikrishnareddymallavarapu/Masters/blob/main/Masters_2_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# Run in python console
import nltk; 
nltk.download('stopwords')

# Run in terminal or command prompt
!python3 -m spacy download en

In [33]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 8.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 76 kB/s 
     |████████████████████████████████| 11.5 MB 236 kB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=0ccb9dc24344d996ada7eecfec685732c5f2141415339e5d4f9ee5b5afff9521
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import spacy

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [5]:

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','trump','donald','say','hillary','clinton','do','united','state','the','day','news','that','have','however','year'])

In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts,bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    nlp = spacy.load('en', disable=['parser', 'ner'])
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
article1 = pd.read_csv('drive/My Drive/Top-Ex/articles1.csv')
article2 = pd.read_csv('drive/My Drive/Top-Ex/articles2.csv')
article3 = pd.read_csv('drive/My Drive/Top-Ex/articles3.csv')

#Concatenating and randomly selecting data

In [9]:
articles = pd.concat([article1,article2,article3], axis=0)
articles = articles.sample(n = 50000, random_state = 2)
articles.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
44480,47352,66559,Trump says his ’massive’ tax plan is coming ne...,Business Insider,Bob Bryan,2017-04-22,2017.0,4.0,NaN,’ ’ ’ President Donald Trump said that he e...
8195,61514,83936,Chicago police have ’no regard’ for minority l...,Fox News,NaN,2016-04-13,2016.0,4.0,https://web.archive.org/web/20160414011639/htt...,Police in Chicago have ”no regard for the san...
10464,113926,168759,A Concussion Can Lead To Sleep Problems That L...,NPR,Jon Hamilton,2016-04-27,2016.0,4.0,http://www.npr.org/sections/health-shots/2016/...,People who sustain a concussion or a more seve...
19198,72606,106668,Trump: I Was Just Joking When I Made Crude Co...,Buzzfeed News,Andrew Kaczynski,2016-03-29,2016.0,3.0,https://web.archive.org/web/20160329002256/htt...,’You can watch the interview here:’] Trump’s c...
17793,71201,102160,"Beyoncé Dropped Out Of Coachella, Because Duh",Buzzfeed News,Alanna Bennett,2017-02-24,2017.0,2.0,https://web.archive.org/web/20170224044447/htt...,"’ It has happened, friends: Beyoncé has offici..."


In [10]:
articles.shape

(50000, 10)

In [11]:
del article1,article2,article3
articles = articles[['id','title','content']]
articles['text']= articles['title']+' '+articles['content']
articles = articles[['id','text']]
articles.columns = ['id','content']

In [14]:
articles.head()

,id,content
44480,66559,Trump says his ’massive’ tax plan is coming ne...
8195,83936,Chicago police have ’no regard’ for minority l...
10464,168759,A Concussion Can Lead To Sleep Problems That L...
19198,106668,Trump: I Was Just Joking When I Made Crude Co...
17793,102160,"Beyoncé Dropped Out Of Coachella, Because Duh..."


In [15]:
articles.dtypes

id          int64
content    object
dtype: object

In [20]:

def createCorpusDict(dataFrame):
    #cleanse the text
    dataFrame.content = dataFrame.content.astype(str)
    data = dataFrame.content.values.tolist()
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]
    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]
    data_words = list(sent_to_words(data))
    
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops,bigram_mod)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en


    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    #cleanse the dictionary
    dictionary = corpora.Dictionary(data_lemmatized)
    # Create Corpus
    texts = data_lemmatized
    # Term Document Frequency
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    return corpus,dictionary,data_lemmatized

In [21]:
corpus, dictionary, data_lemmatized = createCorpusDict(articles)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [27]:
import pickle

filehandler = open("drive/My Drive/Top-Ex/corpus.pkl","wb")
pickle.dump(corpus,filehandler)
filehandler.close()

filehandler = open("drive/My Drive/Top-Ex/dictionary.pkl","wb")
pickle.dump(dictionary,filehandler)
filehandler.close()

filehandler = open("drive/My Drive/Top-Ex/data_lemmatized.pkl","wb")
pickle.dump(data_lemmatized,filehandler)
filehandler.close()

In [23]:
#Train the Topic Modelling
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [28]:
filehandler = open("drive/My Drive/Top-Ex/lda_model_20.pkl","wb")
pickle.dump(lda_model,filehandler)
filehandler.close()

In [24]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
doc_lda

[(0,
  '0.021*"may" + 0.018*"even" + 0.012*"many" + 0.009*"much" + 0.009*"make" + '
  '0.009*"way" + 0.008*"become" + 0.008*"seem" + 0.008*"less" + 0.007*"often"'),
 (1,
  '0.035*"game" + 0.026*"team" + 0.023*"play" + 0.018*"player" + '
  '0.017*"season" + 0.014*"first" + 0.012*"win" + 0.011*"last" + '
  '0.011*"second" + 0.010*"sport"'),
 (2,
  '0.030*"say" + 0.029*"go" + 0.022*"people" + 0.020*"get" + 0.019*"make" + '
  '0.017*"think" + 0.016*"would" + 0.016*"know" + 0.014*"want" + 0.014*"see"'),
 (3,
  '0.022*"city" + 0.016*"car" + 0.012*"build" + 0.012*"home" + 0.010*"new" + '
  '0.009*"building" + 0.009*"store" + 0.009*"local" + 0.008*"also" + '
  '0.008*"driver"'),
 (4,
  '0.018*"use" + 0.018*"email" + 0.014*"post" + 0.013*"medium" + '
  '0.013*"twitter" + 0.013*"story" + 0.012*"write" + 0.012*"information" + '
  '0.011*"account" + 0.009*"network"'),
 (5,
  '0.048*"police" + 0.027*"officer" + 0.026*"say" + 0.021*"kill" + '
  '0.018*"people" + 0.015*"man" + 0.015*"arrest" + 0.015*

In [25]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.028126812418817

Coherence Score:  0.49520995952161784


In [29]:
import pickle
pkl_file = open('drive/My Drive/Top-Ex/data_lemmatized.pkl', 'rb')
data_lemmatized = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('drive/My Drive/Top-Ex/dictionary.pkl', 'rb')
dictionary = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('drive/My Drive/Top-Ex/corpus.pkl', 'rb')
corpus = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('drive/My Drive/Top-Ex/lda_model_20.pkl', 'rb')
lda_model = pickle.load(pkl_file)
pkl_file.close()

In [34]:

import pyLDAvis.gensim
import pickle
import pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


ModuleNotFoundError: ignored

In [ ]:
gamma, _ = lda_model.inference(corpus)
topics = gamma / gamma.sum(axis=1)[:,None]

In [ ]:
topics = pd.DataFrame(topics)
topics.reset_index(drop=True)
topics.rename(columns = lambda x: 'Topic_'+str(x+1).zfill(2), inplace=True)
topics.head()

In [ ]:
finalData = pd.concat([articles,topics], axis=1,sort=False)
finalData.head()

In [ ]:
finalData.head()

In [ ]:
output3 = open('drive/My Drive/Top-Ex/articles_topic_20.pkl', 'wb')
pickle.dump(finalData, output3)
output3.close()

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

CountVec = CountVectorizer(ngram_range=(1,1), # to use bigrams ngram_range=(2,2)
                           stop_words='english')
#transform
Count_data = CountVec.fit_transform()
 
#create dataframe
cv_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())
print(cv_dataframe)